## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\\Weather_Database\WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Makokou,0.5738,12.8642,295.07,93,26,0.65,GA,2021-03-28 19:31:26
1,1,Boda,4.3189,17.4695,300.56,50,52,1.67,CF,2021-03-28 19:31:26
2,2,Foumban,5.7291,10.9001,297.62,39,92,0.83,CM,2021-03-28 19:31:27
3,3,Nola,40.9220,14.5329,285.93,77,0,0.67,IT,2021-03-28 19:31:27
4,4,Oron,4.8256,8.2343,300.55,80,35,2.43,NG,2021-03-28 19:31:27


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = min(city_data_df["Max Temp"]);print(min_temp)
max_temp = max(city_data_df["Max Temp"]);print(max_temp)
user_min_temp = float(input(f"what is the minimum temperaure you would like for your trip (ranges between {min_temp} and {max_temp})?"))
user_max_temp = float(input(f"what is the maximum temperaure you would like for your trip (ranges between {max_temp} and {max_temp})?"))
print(user_min_temp, user_max_temp)


241.46
304.9
242.0 300.0


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_city_df = city_data_df.loc[(city_data_df["Max Temp"] <= user_max_temp) & (city_data_df["Max Temp"] >= user_min_temp)]

preferred_city_df.isna().sum().sum()

0

In [6]:
import numpy as np
# 4a. Determine if there are any empty rows.
# preferred_city_df.count(preferred_city_df.isna)
empty_rows = preferred_city_df.isnull().sum().sum()
print(f"There is/are {empty_rows} row(s) in dataframe 'preferred_city_df'.")

There is/are 0 row(s) in dataframe 'preferred_city_df'.


In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_city_df_cleaned = preferred_city_df.dropna()
preferred_city_df_cleaned

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Makokou,0.5738,12.8642,295.07,93,26,0.65,GA,2021-03-28 19:31:26
2,2,Foumban,5.7291,10.9001,297.62,39,92,0.83,CM,2021-03-28 19:31:27
3,3,Nola,40.9220,14.5329,285.93,77,0,0.67,IT,2021-03-28 19:31:27
6,6,Paoua,7.2427,16.4406,299.27,53,63,2.24,CF,2021-03-28 19:31:28
14,14,Mao,39.8885,4.2658,285.15,87,0,3.09,ES,2021-03-28 19:31:30
...,...,...,...,...,...,...,...,...,...,...
168,168,Cherskiy,68.7500,161.3000,250.22,93,100,0.33,RU,2021-03-28 19:32:09
169,169,Srednekolymsk,67.4500,153.6833,248.30,99,100,2.13,RU,2021-03-28 19:32:10
170,170,Anadyr,64.7500,177.4833,263.15,79,75,8.00,RU,2021-03-28 19:32:10
171,171,Kachug,53.9606,105.8817,264.89,96,100,0.51,RU,2021-03-28 19:32:10


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_city_df_cleaned[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df["Current Description"] = ""
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name,Current Description
0,Makokou,GA,295.07,0.5738,12.8642,,
2,Foumban,CM,297.62,5.7291,10.9001,,
3,Nola,IT,285.93,40.9220,14.5329,,
6,Paoua,CF,299.27,7.2427,16.4406,,
14,Mao,ES,285.15,39.8885,4.2658,,
...,...,...,...,...,...,...,...
168,Cherskiy,RU,250.22,68.7500,161.3000,,
169,Srednekolymsk,RU,248.30,67.4500,153.6833,,
170,Anadyr,RU,263.15,64.7500,177.4833,,
171,Kachug,RU,264.89,53.9606,105.8817,,


In [9]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import *

# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": ""
}

# 6b. Iterate through the hotel DataFrame.

# 6c. Get latitude and longitude from DataFrame
lats =  preferred_city_df["Lat"]
longs = preferred_city_df["Lng"]
hotels_found = dict()
# coordinates = zip(lats, longs)
# coordinates_list = list(coordinates)
for i, row in hotel_df.iterrows():
    lat = row["Lat"]
    longit = row["Lng"]
    params["location"] = f"{lat}, {longit}"
    # print(params)
# 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    
    a_hotel = requests.get(base_url, params=params).json()
    # print(base_url)
# 6e. Make request and retrieve the JSON data from the search.
    try:
# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        a_hotel_name = a_hotel["results"][0]["name"]
        hotel_df.at[i, "Hotel Name"] = a_hotel_name
    except(IndexError):
        # print("Hotel not found...skipping.")
        pass
    i+= 1

        

In [10]:
# len(hotels_found)
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name,Current Description
0,Makokou,GA,295.07,0.5738,12.8642,WAMY RESIDENCE HOTEL,
2,Foumban,CM,297.62,5.7291,10.9001,Hotel Pekassa De Karche,
3,Nola,IT,285.93,40.9220,14.5329,Pamaran Hotel,
6,Paoua,CF,299.27,7.2427,16.4406,Quartier Paoua Elien,
14,Mao,ES,285.15,39.8885,4.2658,ARTIEM Capri,
...,...,...,...,...,...,...,...
168,Cherskiy,RU,250.22,68.7500,161.3000,Gostinitsa,
169,Srednekolymsk,RU,248.30,67.4500,153.6833,,
170,Anadyr,RU,263.15,64.7500,177.4833,"Гостиница ""Анадырь""",
171,Kachug,RU,264.89,53.9606,105.8817,"The hotel ""Empire""",


In [11]:
import pandas as pd
import numpy as np
hotel_df = hotel_df.replace(r'^\s*$', np.nan, regex=True)
hotel_df
# 7. Drop the rows where there is no Hotel Name.
empty_rows = hotel_df["Hotel Name"].isna() #.sum().sum()

hotel_df = hotel_df[hotel_df['Hotel Name'].notna()]
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name,Current Description
0,Makokou,GA,295.07,0.5738,12.8642,WAMY RESIDENCE HOTEL,NaN
2,Foumban,CM,297.62,5.7291,10.9001,Hotel Pekassa De Karche,NaN
3,Nola,IT,285.93,40.9220,14.5329,Pamaran Hotel,NaN
6,Paoua,CF,299.27,7.2427,16.4406,Quartier Paoua Elien,NaN
14,Mao,ES,285.15,39.8885,4.2658,ARTIEM Capri,NaN
...,...,...,...,...,...,...,...
167,Deputatskiy,RU,255.51,69.3000,139.9000,Baza Otdykha,NaN
168,Cherskiy,RU,250.22,68.7500,161.3000,Gostinitsa,NaN
170,Anadyr,RU,263.15,64.7500,177.4833,"Гостиница ""Анадырь""",NaN
171,Kachug,RU,264.89,53.9606,105.8817,"The hotel ""Empire""",NaN


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "..\Weather_Database\weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
0,0.5738,12.8642
2,5.7291,10.9001
3,40.9220,14.5329
6,7.2427,16.4406
14,39.8885,4.2658
...,...,...
167,69.3000,139.9000
168,68.7500,161.3000
170,64.7500,177.4833
171,53.9606,105.8817


In [14]:
# 11a. Add a marker layer for each city to the map. 
import gmaps
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
max_temp = hotel_df["Max Temp"]

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))